# My code

بارگیری فایل دیتاست از آدرس معرفی شده

In [5]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip

--2023-06-21 14:16:47--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1         100%[===================>]   5.64M  19.6MB/s    in 0.3s    

2023-06-21 14:16:47 (19.6 MB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]



باز کردن فایل مذکور از حالت فشرده

In [6]:
!unzip ml-1m.zip

Archive:  ml-1m.zip
replace ml-1m/movies.dat? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

فراخوانی کتابخانه ها - پکیج ها و توابع لازم

In [3]:
import numpy as np
from collections import defaultdict
from sklearn.model_selection import train_test_split

data_path = 'ml-1m/ratings.dat'
n_users = 6040
n_movies = 3706



<div dir=rtl>
خواندن فایل rating مربوط به نظرات کاربران در مورد فیلم ها (آی دی کاربر - آی دی فیلم - نمره ی داده شده)

In [10]:
# UserID::MovieID::Rating
data_path = 'ml-1m/ratings.dat'
rate_data = []
with open(data_path, 'r') as file:
  for line in file.readlines():
    # print(line)
    user_id, movie_id, rating, _ = line.split("::")
    rating = int(rating)
    user_id = int(user_id)
    movie_id = int(movie_id)
    rate_data.append([user_id, movie_id, rating])

print(rate_data[:10])
total_rating_record = len(rate_data)
print("total_rating_record: " + str(total_rating_record))

[[1, 1193, 5], [1, 661, 3], [1, 914, 3], [1, 3408, 4], [1, 2355, 5], [1, 1197, 3], [1, 1287, 5], [1, 2804, 5], [1, 594, 4], [1, 919, 4]]
total_rating_record: 1000209


<div dir=rtl>
خواندن فایل movies مربوط به اطلاعات فیلم ها (آی دی - عنوان - ژانر)


In [11]:
# MovieID::Title::Genres
data_path = 'ml-1m/movies.dat'
movie_data = []
with open(data_path, 'r', encoding = "ISO-8859-1") as file:
  for line in file.readlines():
    movie_id, title, genres = line.split("::")
    movie_id = int(movie_id)
    genres = genres.split('\n')[0]
    movie_data.append([movie_id, title, genres])

print(movie_data[:10])
print(len(movie_data))
max_movie_id = max(movie_data)[0] #جهت دريافت بزرگترين آي. دي از ديتا ست فيلم ها
print(max(movie_data))
print(max_movie_id)

[[1, 'Toy Story (1995)', "Animation|Children's|Comedy"], [2, 'Jumanji (1995)', "Adventure|Children's|Fantasy"], [3, 'Grumpier Old Men (1995)', 'Comedy|Romance'], [4, 'Waiting to Exhale (1995)', 'Comedy|Drama'], [5, 'Father of the Bride Part II (1995)', 'Comedy'], [6, 'Heat (1995)', 'Action|Crime|Thriller'], [7, 'Sabrina (1995)', 'Comedy|Romance'], [8, 'Tom and Huck (1995)', "Adventure|Children's"], [9, 'Sudden Death (1995)', 'Action'], [10, 'GoldenEye (1995)', 'Action|Adventure|Thriller']]
3883
[3952, 'Contender, The (2000)', 'Drama|Thriller']
3952


تنظیم کردن آی دی فیلم با اندیس آرایه

In [12]:
movie_data_2 = [i for i in range(max_movie_id)]
for i in movie_data:
  index = i[0]
  movie_data_2[index-1] = i

print(movie_data_2[:10])
print(movie_data_2[-1])


[[1, 'Toy Story (1995)', "Animation|Children's|Comedy"], [2, 'Jumanji (1995)', "Adventure|Children's|Fantasy"], [3, 'Grumpier Old Men (1995)', 'Comedy|Romance'], [4, 'Waiting to Exhale (1995)', 'Comedy|Drama'], [5, 'Father of the Bride Part II (1995)', 'Comedy'], [6, 'Heat (1995)', 'Action|Crime|Thriller'], [7, 'Sabrina (1995)', 'Comedy|Romance'], [8, 'Tom and Huck (1995)', "Adventure|Children's"], [9, 'Sudden Death (1995)', 'Action'], [10, 'GoldenEye (1995)', 'Action|Adventure|Thriller']]
[3952, 'Contender, The (2000)', 'Drama|Thriller']


<div dir=rtl>
خواندن فایل users مربوط به اطلاعات کاربران (آی دی کاربر - جنسیت - سن - شغل - کدپستی)

In [13]:
# UserID::Gender::Age::Occupation::Zip-code
data_path = 'ml-1m/users.dat'
user_data = []
with open(data_path, 'r') as file:
  for line in file.readlines():
    # print(line)
    user_id, gender, age, occupation, zip_code = line.split("::")
    user_id = int(user_id)
    age = int(age)
    zip_code = zip_code.split("\n")[0]
    user_data.append([user_id, gender, age, occupation, zip_code])

print(user_data[:10])
print(len(user_data))
print(max(user_data))

[[1, 'F', 1, '10', '48067'], [2, 'M', 56, '16', '70072'], [3, 'M', 25, '15', '55117'], [4, 'M', 45, '7', '02460'], [5, 'M', 25, '20', '55455'], [6, 'F', 50, '9', '55117'], [7, 'M', 35, '1', '06810'], [8, 'M', 25, '12', '11413'], [9, 'M', 25, '17', '61614'], [10, 'F', 35, '1', '95370']]
6040
[6040, 'M', 25, '6', '11106']


ایجاد یک ليست جدید برای ايجاد رابطه كليه ي اطلاعات

In [ ]:
collective_data = []
collective_data = rate_data.copy()
print(collective_data[:10])

[[1, 1193, 5], [1, 661, 3], [1, 914, 3], [1, 3408, 4], [1, 2355, 5], [1, 1197, 3], [1, 1287, 5], [1, 2804, 5], [1, 594, 4], [1, 919, 4]]


In [ ]:
print(user_data[0])

[1, 'F', 1, '10', '48067']


افزودن اطلاعات کاربران و فیلم ها به دیتاست نظرات

پس از اجرای این بخش -در کنار هر نظر- مشخصات فرد نظر دهنده و همچنین ژانر فیلم نیز افزوده می گردد

In [14]:
collective_data = []
collective_data = rate_data.copy()
print(collective_data[:10])

#َافزودن اطلاعات كاربران به جدول ارزيابي
for i in range(len(collective_data)):
# for i in range(10):
  user_id = rate_data[i][0]
  # print(user_id)
  movie_id = rate_data[i][1]
  # print(movie_id)
  rating = rate_data[i][2]
  # print(rating)

  ############## all features selected from user data (remove zip_code col)
  temp = user_data[user_id - 1][:4].copy()
  #######################################

  # print(1)
  # print(temp)
  temp.append(movie_id)
  # print(2)
  # print(temp)
  temp.append(rating)
  # print(3)
  # print(temp)
  collective_data[i] = temp

print(collective_data[:10])

#َافزودن اطلاعات ويدئوها به جدول ارزيابي
for i in range(len(collective_data)):
# for i in range(10):
  ######## all features selected from user data + movie_id
  temp = collective_data[i][:5]
  ###########################################
  # print(temp)
  movie_id = collective_data[i][4]  ####################
  # print(movie_id)
  rating = collective_data[i][5]  #####################
  # print(rating)
  genres = movie_data_2[movie_id - 1][2]  ########### feature selected genres
  temp.append(genres)
  temp.append(rating)

  collective_data[i] = temp

print(collective_data[:10])
#UserID::Gender::Age::Occupation::MovieID::Genres::Rating


[[1, 1193, 5], [1, 661, 3], [1, 914, 3], [1, 3408, 4], [1, 2355, 5], [1, 1197, 3], [1, 1287, 5], [1, 2804, 5], [1, 594, 4], [1, 919, 4]]
[[1, 'F', 1, '10', 1193, 5], [1, 'F', 1, '10', 661, 3], [1, 'F', 1, '10', 914, 3], [1, 'F', 1, '10', 3408, 4], [1, 'F', 1, '10', 2355, 5], [1, 'F', 1, '10', 1197, 3], [1, 'F', 1, '10', 1287, 5], [1, 'F', 1, '10', 2804, 5], [1, 'F', 1, '10', 594, 4], [1, 'F', 1, '10', 919, 4]]
[[1, 'F', 1, '10', 1193, 'Drama', 5], [1, 'F', 1, '10', 661, "Animation|Children's|Musical", 3], [1, 'F', 1, '10', 914, 'Musical|Romance', 3], [1, 'F', 1, '10', 3408, 'Drama', 4], [1, 'F', 1, '10', 2355, "Animation|Children's|Comedy", 5], [1, 'F', 1, '10', 1197, 'Action|Adventure|Comedy|Romance', 3], [1, 'F', 1, '10', 1287, 'Action|Adventure|Drama', 5], [1, 'F', 1, '10', 2804, 'Comedy|Drama', 5], [1, 'F', 1, '10', 594, "Animation|Children's|Musical", 4], [1, 'F', 1, '10', 919, "Adventure|Children's|Drama|Musical", 4]]


تقسیم کردن داده ها به دو بخش داده های آموزش مدل و داده‌های تست و ارزيابي مدل

 مطابق معمول این دو به نسبت ۸۰-۲۰ تقسیم شده اند.

In [18]:
collective_data_train, collective_data_test = train_test_split(collective_data,
                                                    test_size=0.2,
                                                    random_state=80)
#Using from sklearn.model_selection import train_test_split
# معمولا داده ها را به دوقسمت آموزش و تست تقسيم مي كنيم
# نسبت در اين  بخش
# 0.8 آموزش
# 0.2 تست

# با داده آموزش مدل را آموزش مي دهيم
# با داده تست مدل را ارزيابي مي كنيم

print(collective_data_train[:10])
print("Training data length:")
print(len(collective_data_train))

print(collective_data_test[:10])
print("Test data length:")
print(len(collective_data_test))

[[5664, 'M', 18, '4', 3275, 'Action|Comedy', 4], [2820, 'F', 35, '0', 160, 'Action|Adventure|Mystery|Sci-Fi', 2], [5721, 'M', 45, '11', 2021, 'Fantasy|Sci-Fi', 3], [3579, 'M', 18, '0', 1597, 'Action|Mystery|Romance|Thriller', 3], [3565, 'M', 25, '18', 671, 'Comedy|Sci-Fi', 2], [3748, 'M', 18, '4', 1961, 'Drama', 5], [1052, 'F', 35, '2', 3244, 'Comedy|Romance', 3], [1202, 'M', 35, '12', 799, 'Comedy|Horror', 2], [479, 'M', 25, '12', 1011, "Adventure|Children's|Comedy", 3], [2097, 'M', 18, '5', 3054, "Animation|Children's", 3]]
Training data length:
800167
[[4957, 'M', 25, '2', 3545, 'Musical|War', 5], [5857, 'F', 45, '2', 2628, 'Action|Adventure|Fantasy|Sci-Fi', 5], [4095, 'M', 25, '0', 3578, 'Action|Drama', 4], [4685, 'M', 35, '1', 2336, 'Drama', 2], [1258, 'M', 45, '7', 296, 'Crime|Drama', 5], [3808, 'M', 25, '7', 2240, 'Drama', 4], [692, 'F', 25, '0', 24, 'Drama|Sci-Fi', 3], [5088, 'M', 25, '17', 750, 'Sci-Fi|War', 5], [2104, 'M', 45, '15', 924, 'Drama|Mystery|Sci-Fi|Thriller', 2], [

 شمارش نظرات کاربران به نسبت علاقه مندی یا عدم علاقه مندی به فیلم ها

In [22]:
# find repeated user ID
# باتوجه به اينكه در محاسبات نايو بيز به 3 داده ذيل احتياج داريم آنها را در ديكشنري نگهداري مي كنيم
user_id_count = {}
count = 0
print(user_id_count)
for i in collective_data_train:
  # i[0]=> UserId
  if i[0] not in user_id_count:  ##########
    user_id_count[i[0]] = {'count':1, 'y':0 , 'n':0}  ###############
  else:
    user_id_count[i[0]]['count'] = user_id_count[i[0]]['count'] + 1  ##########
    count += 1

  # i[6]=> rate
  if i[6] >= 4:
    user_id_count[i[0]]['y'] = user_id_count[i[0]]['y'] + 1  ##################
  else:
    user_id_count[i[0]]['n'] = user_id_count[i[0]]['n'] + 1  ##################

print(user_id_count)
print("User count:"+ str(len(user_id_count)))

{}
{5664: {'count': 232, 'y': 171, 'n': 61}, 2820: {'count': 724, 'y': 210, 'n': 514}, 5721: {'count': 32, 'y': 24, 'n': 8}, 3579: {'count': 212, 'y': 74, 'n': 138}, 3565: {'count': 119, 'y': 44, 'n': 75}, 3748: {'count': 28, 'y': 17, 'n': 11}, 1052: {'count': 51, 'y': 45, 'n': 6}, 1202: {'count': 409, 'y': 236, 'n': 173}, 479: {'count': 77, 'y': 50, 'n': 27}, 2097: {'count': 133, 'y': 56, 'n': 77}, 2967: {'count': 423, 'y': 211, 'n': 212}, 4725: {'count': 787, 'y': 160, 'n': 627}, 4328: {'count': 253, 'y': 161, 'n': 92}, 2272: {'count': 107, 'y': 76, 'n': 31}, 4041: {'count': 686, 'y': 284, 'n': 402}, 5636: {'count': 683, 'y': 264, 'n': 419}, 1988: {'count': 532, 'y': 289, 'n': 243}, 3308: {'count': 581, 'y': 237, 'n': 344}, 757: {'count': 360, 'y': 139, 'n': 221}, 368: {'count': 251, 'y': 163, 'n': 88}, 2126: {'count': 70, 'y': 20, 'n': 50}, 3473: {'count': 130, 'y': 92, 'n': 38}, 6000: {'count': 401, 'y': 139, 'n': 262}, 2071: {'count': 201, 'y': 94, 'n': 107}, 2125: {'count': 192, 

شمارش کاربران به نسبت علاقه مندی یا عدم علاقه مندی به فیلم ها

In [23]:
# find repeated gender
gender_count = {}
count = 0
print(gender_count)
for i in collective_data_train:
  # i[1]=> User Gender
  if i[1] not in gender_count:  ##########
    gender_count[i[1]] = {'count':1, 'y':0 , 'n':0}  ###############
  else:
    gender_count[i[1]]['count'] = gender_count[i[1]]['count'] + 1  ###########
    count += 1

  # i[6]=> rate
  if i[6] >= 4:
    gender_count[i[1]]['y'] = gender_count[i[1]]['y'] + 1  ##################
  else:
    gender_count[i[1]]['n'] = gender_count[i[1]]['n'] + 1  ##################


print("Gender dict:")
print(count)
print(gender_count)
print("Gender type count:")
print(len(gender_count))

{}
Gender dict:
800165
{'M': {'count': 602850, 'y': 343581, 'n': 259269}, 'F': {'count': 197317, 'y': 116642, 'n': 80675}}
Gender type count:
2


شمارش سن کاربران به نسبت علاقه مندی یا عدم علاقه مندی به فیلم ها

In [24]:
# find repeated Age
age_count = {}
count = 0
print(age_count)
for i in collective_data_train:
  # i[2]=> User Age
  if i[2] not in age_count:  ##########
    age_count[i[2]] = {'count':1, 'y':0 , 'n':0}  ###############
  else:
    age_count[i[2]]['count'] = age_count[i[2]]['count'] + 1  ###############
    count += 1

  # i[6]=> rate
  if i[6] >= 4:
    age_count[i[2]]['y'] = age_count[i[2]]['y'] + 1  ##################
  else:
    age_count[i[2]]['n'] = age_count[i[2]]['n'] + 1  ##################



print("Age dict:")
print(count)
print(age_count)
print("Age type count:")
print(len(age_count))

{}
Age dict:
800160
{18: {'count': 146808, 'y': 80462, 'n': 66346}, 35: {'count': 159311, 'y': 93481, 'n': 65830}, 45: {'count': 66923, 'y': 39647, 'n': 27276}, 25: {'count': 316378, 'y': 178106, 'n': 138272}, 50: {'count': 58039, 'y': 36032, 'n': 22007}, 56: {'count': 30936, 'y': 20000, 'n': 10936}, 1: {'count': 21772, 'y': 12495, 'n': 9277}}
Age type count:
7


شمارش شغل کاربران به نسبت علاقه مندی یا عدم علاقه مندی به فیلم ها

In [33]:
# find repeated occupation
occupation_count = {}
count = 0
print(occupation_count)
for i in collective_data_train:
  # i[3]=> User Occupation
  if i[3] not in occupation_count:  ##########
    occupation_count[i[3]] = {'count':1, 'y':0 , 'n':0}  ###############
  else:
    occupation_count[i[3]]['count'] = occupation_count[i[3]]['count'] + 1  ####
    count += 1

  # i[6]=> rate
  if i[6] >= 4:
    occupation_count[i[3]]['y'] = occupation_count[i[3]]['y'] + 1  ###########
  else:
    occupation_count[i[3]]['n'] = occupation_count[i[3]]['n'] + 1  ###########


print("occupation dict:")
print(count)
print(occupation_count)
print("occupation count:")
print(len(occupation_count))

{}
occupation dict:
800146
{'4': {'count': 104791, 'y': 58790, 'n': 46001}, '0': {'count': 104445, 'y': 57928, 'n': 46517}, '11': {'count': 16377, 'y': 9819, 'n': 6558}, '18': {'count': 9747, 'y': 5260, 'n': 4487}, '2': {'count': 40228, 'y': 23014, 'n': 17214}, '12': {'count': 45762, 'y': 27604, 'n': 18158}, '5': {'count': 17553, 'y': 9511, 'n': 8042}, '3': {'count': 25358, 'y': 15294, 'n': 10064}, '7': {'count': 84520, 'y': 49156, 'n': 35364}, '1': {'count': 68169, 'y': 39376, 'n': 28793}, '20': {'count': 48299, 'y': 26469, 'n': 21830}, '14': {'count': 39389, 'y': 23164, 'n': 16225}, '16': {'count': 36876, 'y': 21262, 'n': 15614}, '17': {'count': 58074, 'y': 33812, 'n': 24262}, '6': {'count': 29607, 'y': 18140, 'n': 11467}, '13': {'count': 10936, 'y': 7090, 'n': 3846}, '10': {'count': 18639, 'y': 10595, 'n': 8044}, '19': {'count': 11954, 'y': 6144, 'n': 5810}, '15': {'count': 18268, 'y': 11331, 'n': 6937}, '9': {'count': 9030, 'y': 5335, 'n': 3695}, '8': {'count': 2145, 'y': 1129, 'n'

شمارش کاربران به نسبت علاقه مندی یا عدم علاقه مندی به هر فیلم

In [26]:
# find repeated movie ID
movie_id_count = {}
count = 0
print(movie_id_count)
for i in collective_data_train:
  # i[4]=> movieId
  if i[4] not in movie_id_count:  ##########
    movie_id_count[i[4]] = {'count':1, 'y':0 , 'n':0}  ###############
  else:
    movie_id_count[i[4]]['count']  = movie_id_count[i[4]]['count']  + 1  ######
    count += 1

  # i[6]=> rate
  if i[6] >= 4:
    movie_id_count[i[4]]['y'] = movie_id_count[i[4]]['y'] + 1  ###########
  else:
    movie_id_count[i[4]]['n'] = movie_id_count[i[4]]['n'] + 1  ###########


print("Reapeted movie ID count:")
print(count)
print(movie_id_count)
print("Movies count:")
print(len(movie_id_count))

{}
Reapeted movie ID count:
796494
{3275: {'count': 66, 'y': 42, 'n': 24}, 160: {'count': 461, 'y': 50, 'n': 411}, 2021: {'count': 614, 'y': 294, 'n': 320}, 1597: {'count': 668, 'y': 332, 'n': 336}, 671: {'count': 366, 'y': 231, 'n': 135}, 1961: {'count': 1034, 'y': 809, 'n': 225}, 3244: {'count': 259, 'y': 163, 'n': 96}, 799: {'count': 270, 'y': 131, 'n': 139}, 1011: {'count': 119, 'y': 23, 'n': 96}, 3054: {'count': 108, 'y': 15, 'n': 93}, 1131: {'count': 183, 'y': 160, 'n': 23}, 185: {'count': 442, 'y': 116, 'n': 326}, 21: {'count': 1091, 'y': 634, 'n': 457}, 1136: {'count': 1301, 'y': 1120, 'n': 181}, 3300: {'count': 425, 'y': 206, 'n': 219}, 1193: {'count': 1336, 'y': 1170, 'n': 166}, 1562: {'count': 478, 'y': 71, 'n': 407}, 1097: {'count': 1843, 'y': 1340, 'n': 503}, 1912: {'count': 723, 'y': 473, 'n': 250}, 3243: {'count': 235, 'y': 39, 'n': 196}, 1206: {'count': 970, 'y': 721, 'n': 249}, 898: {'count': 467, 'y': 407, 'n': 60}, 168: {'count': 275, 'y': 84, 'n': 191}, 3791: {'coun

شمارش کاربران به نسبت علاقه مندی یا عدم علاقه مندی به ژانر ها



In [27]:
# find repeated genres
genres_count = {}
count = 0
print(genres_count)
for i in collective_data_train:

  # i[5]=> movieGenres
  if i[5] not in genres_count:  ##########
    genres_count[i[5]] = {'count':1, 'y':0 , 'n':0}  ###############
  else:
    genres_count[i[5]]['count'] = genres_count[i[5]]['count'] + 1  ############

  # i[6]=> rate
  if i[6] >= 4:
    genres_count[i[5]]['y'] = genres_count[i[5]]['y'] + 1  ##################
  else:
    genres_count[i[5]]['n'] = genres_count[i[5]]['n'] + 1  ##################

    count += 1

print("Reapeted genres count:")
print(count)
print(genres_count)
print("Genres type count:")
print(len(genres_count))

{}
Reapeted genres count:
339944
{'Action|Comedy': {'count': 3342, 'y': 988, 'n': 2354}, 'Action|Adventure|Mystery|Sci-Fi': {'count': 461, 'y': 50, 'n': 411}, 'Fantasy|Sci-Fi': {'count': 614, 'y': 294, 'n': 320}, 'Action|Mystery|Romance|Thriller': {'count': 668, 'y': 332, 'n': 336}, 'Comedy|Sci-Fi': {'count': 7466, 'y': 4226, 'n': 3240}, 'Drama': {'count': 89184, 'y': 58143, 'n': 31041}, 'Comedy|Romance': {'count': 34204, 'y': 18673, 'n': 15531}, 'Comedy|Horror': {'count': 7734, 'y': 3805, 'n': 3929}, "Adventure|Children's|Comedy": {'count': 653, 'y': 155, 'n': 498}, "Animation|Children's": {'count': 7091, 'y': 3876, 'n': 3215}, 'Sci-Fi|Thriller': {'count': 3739, 'y': 1332, 'n': 2407}, 'Action|Comedy|Drama': {'count': 1424, 'y': 822, 'n': 602}, 'Comedy': {'count': 93556, 'y': 49941, 'n': 43615}, 'Action|Sci-Fi': {'count': 11422, 'y': 5082, 'n': 6340}, 'Action|Adventure|Crime': {'count': 1471, 'y': 501, 'n': 970}, "Children's|Drama|Fantasy|Sci-Fi": {'count': 1843, 'y': 1340, 'n': 503}, 

In [28]:
# # find repeated genres
# s = set()
# count = 0
# print(s)
# for i in collective_data_train:
#   if i[5] not in s:  ##########
#     s.add(i[5])
#   else:
#     count += 1


# print("Reapeted genres count:")
# print(count)

شمارش نظرات کاربران به نسبت علاقه مندی یا عدم علاقه مندی به فیلم ها

In [29]:
# find repeated Rating
rating_count = {}
count = 0
print(rating_count)
for i in collective_data_train:

  # i[6]=> rate
  if i[6] not in rating_count:  ##########
    rating_count[i[6]] = 1  ###############
  else:
    rating_count[i[6]] = rating_count[i[6]] + 1  ###################
    count += 1

print("Rating dict:")
print(count)
print(rating_count)
print("Rating type count:")
print(len(rating_count))

{}
Rating dict:
800162
{4: 279312, 2: 86124, 3: 208887, 5: 180911, 1: 44933}
Rating type count:
5


نمایش داده های بدست آمده تا کنون

In [30]:
#UserID::Gender::Age::Occupation::MovieID::Genres::Rating
#UserID=i[0]::Gender=i[1]::Age=i[2]::Occupation=i[3]::MovieId=i[4]::Genres=i[5]::Rating=i[6]
print(collective_data[:10])
print(collective_data_test[:10])
print(collective_data_train[:10])

[[1, 'F', 1, '10', 1193, 'Drama', 5], [1, 'F', 1, '10', 661, "Animation|Children's|Musical", 3], [1, 'F', 1, '10', 914, 'Musical|Romance', 3], [1, 'F', 1, '10', 3408, 'Drama', 4], [1, 'F', 1, '10', 2355, "Animation|Children's|Comedy", 5], [1, 'F', 1, '10', 1197, 'Action|Adventure|Comedy|Romance', 3], [1, 'F', 1, '10', 1287, 'Action|Adventure|Drama', 5], [1, 'F', 1, '10', 2804, 'Comedy|Drama', 5], [1, 'F', 1, '10', 594, "Animation|Children's|Musical", 4], [1, 'F', 1, '10', 919, "Adventure|Children's|Drama|Musical", 4]]
[[4957, 'M', 25, '2', 3545, 'Musical|War', 5], [5857, 'F', 45, '2', 2628, 'Action|Adventure|Fantasy|Sci-Fi', 5], [4095, 'M', 25, '0', 3578, 'Action|Drama', 4], [4685, 'M', 35, '1', 2336, 'Drama', 2], [1258, 'M', 45, '7', 296, 'Crime|Drama', 5], [3808, 'M', 25, '7', 2240, 'Drama', 4], [692, 'F', 25, '0', 24, 'Drama|Sci-Fi', 3], [5088, 'M', 25, '17', 750, 'Sci-Fi|War', 5], [2104, 'M', 45, '15', 924, 'Drama|Mystery|Sci-Fi|Thriller', 2], [3756, 'M', 18, '12', 1863, 'Comedy', 

محاسبه ی تعداد فیلم های مورد علاقه و غیر مورد علاقه توسط نظرات کاربران

In [31]:
y_n_rate = {}
y_n_rate['Y'] = rating_count[5] + rating_count[4]
y_n_rate['N'] = rating_count[3] + rating_count[2] + rating_count[1]
y_n_rate['T'] = y_n_rate['Y'] + y_n_rate['N']
# y_n_rate['Y/T'] = y_n_rate['Y'] / y_n_rate['T']
# y_n_rate['N/T'] = y_n_rate['N'] / y_n_rate['T']

print(y_n_rate)

{'Y': 460223, 'N': 339944, 'T': 800167}


محاسبه ی قسمت های مختلف فرمول

و در نهایت قضاوت در مورد میزان دقت مدل

In [32]:
total_test_data_count = 0
successful_test_data_count = 0
failure_test_data_count = 0
for i in collective_data_test:
  total_test_data_count += 1

  # Yes
  p1 = (user_id_count[i[0]]['y'] + 1) /  (y_n_rate['Y'] + 2)
  p2 = (gender_count[i[1]]['y'] + 1) /  (y_n_rate['Y']+ 2)
  p3 = (age_count[i[2]]['y'] + 1) /  (y_n_rate['Y'] + 2)
  p4 = (occupation_count[i[3]]['y'] + 1) /  (y_n_rate['Y'] + 2)
  try:
    p5 = (movie_id_count[i[4]]['y'] + 1) /  (y_n_rate['Y'] + 2)
  except:
    p5 = 1/2
  p6 = (genres_count[i[5]]['y'] + 1) /  (y_n_rate['Y'] + 2)

  p_yes = (y_n_rate['Y']) /  (y_n_rate['T'])

  p_x_yes = p1*p2*p3*p4*p5*p6*p_yes




  # NO
  p1 = (user_id_count[i[0]]['y'] + 1) /  (y_n_rate['Y'] + 2)
  p2 = (gender_count[i[1]]['y'] + 1) /  (y_n_rate['Y']+ 2)
  p3 = (age_count[i[2]]['y'] + 1) /  (y_n_rate['Y'] + 2)
  p4 = (occupation_count[i[3]]['y'] + 1) /  (y_n_rate['Y'] + 2)
  try:
    p5 = (movie_id_count[i[4]]['y'] + 1) /  (y_n_rate['Y'] + 2)
  except:
    p5 = 1/2
  p6 = (genres_count[i[5]]['y'] + 1) /  (y_n_rate['Y'] + 2)

  p_no = (y_n_rate['N']) /  (y_n_rate['T'])

  p_x_no = p1*p2*p3*p4*p5*p6*p_no



  p1 =  (user_id_count[i[0]]['count'] + 1) / (total_rating_record + 2)
  p2 =  (gender_count[i[1]]['count'] + 1) / (total_rating_record + 2)
  p3 =  (age_count[i[2]]['count'] + 1) / (total_rating_record + 2)
  p4 =  (occupation_count[i[3]]['count'] + 1) / (total_rating_record + 2)
  try:
    p5 =  (movie_id_count[i[4]]['count'] + 1) / (total_rating_record + 2)
  except:
    p5 = 1/(total_rating_record + 2)
  p6 =  (genres_count[i[5]]['count'] + 1) / (total_rating_record + 2)

  p_x = p1*p2*p3*p4*p5*p6


  p_no_x = p_x_no * p_no / p_x

  p_yes_x = p_x_yes * p_yes / p_x

  # if total_test_data_count < 100:
  #   print(str(total_test_data_count)+": P yes=" + str(p_yes_x))
  #   print(str(total_test_data_count)+": P no=" + str(p_no_x))

  if p_yes_x > p_no_x:
    if i[6] >= 4:
      successful_test_data_count += 1
    else:
      failure_test_data_count += 1
  elif p_yes_x <= p_no_x:
    if i[6]< 4:
      successful_test_data_count += 1
    else:
      failure_test_data_count += 1

print("Successful prediction count:")
print(successful_test_data_count)
print("Failure prediction count:")
print(failure_test_data_count)
print("Model accuarity:")
print(successful_test_data_count / total_test_data_count)

Successful prediction count:
115058
Failure prediction count:
84984
Model accuarity:
0.5751692144649624


In [ ]:
print(list2[:10])

[0.03610613777762338, 0.09738004744988936, 0.016339023245621403, 0.0444026635873561, 0.018404526599664177, 0.040070496605138534, 0.004390067887410558, 0.019411038867987666, 0.017753617645713458, 0.017976667536251923]


# Book code

In [ ]:
def load_rating_data(data_path, n_users, n_movies):
  """
  Load rating data from file and also return the number of
    ratings for each movie and movie_id index mapping
  @param data_path: path of the rating data file
  @param n_users: number of users
  @param n_movies: number of movies that have ratings
  @return: rating data in the numpy array of [user, movie];
        movie_n_rating, {movie_id: number of ratings};
        movie_id_mapping, {movie_id: column index in
        rating data}
  """
  data = np.zeros([n_users, n_movies], dtype=np.float32)
  movie_id_mapping = {}
  movie_n_rating = defaultdict(int)
  with open(data_path, 'r') as file:
    for line in file.readlines()[1:]:
      user_id, movie_id, rating, _ = line.split("::")
      user_id = int(user_id) - 1
      if movie_id not in movie_id_mapping:
        movie_id_mapping[movie_id] = len(movie_id_mapping)
      rating = int(rating)
      data[user_id, movie_id_mapping[movie_id]] = rating

      if rating > 0:
          movie_n_rating[movie_id] += 1
  return data, movie_n_rating, movie_id_mapping

In [ ]:
data, movie_n_rating, movie_id_mapping = load_rating_data(data_path, n_users, n_movies)

In [ ]:
def display_distribution(data):
  values, counts = np.unique(data, return_counts=True)
  for value, count in zip(values, counts):
    print(f'Number of rating {int(value)}: {count}')

display_distribution(data)

Number of rating 0: 21384032
Number of rating 1: 56174
Number of rating 2: 107557
Number of rating 3: 261197
Number of rating 4: 348971
Number of rating 5: 226309


In [ ]:
movie_id_most, n_rating_most = sorted(movie_n_rating.items(),
                                      key=lambda d: d[1], reverse=True)[0]
print(f'Movie ID {movie_id_most} has {n_rating_most} ratings.')

Movie ID 2858 has 3428 ratings.


In [ ]:
X_raw = np.delete(data, movie_id_mapping[movie_id_most], axis=1)
Y_raw = data[:, movie_id_mapping[movie_id_most]]

In [ ]:
X = X_raw[Y_raw > 0]
Y = Y_raw[Y_raw > 0]
print('Shape of X:', X.shape)

Shape of X: (3428, 3705)


In [ ]:
print('Shape of Y:', Y.shape)

Shape of Y: (3428,)


In [ ]:
display_distribution(Y)

Number of rating 1: 83
Number of rating 2: 134
Number of rating 3: 358
Number of rating 4: 890
Number of rating 5: 1963


In [ ]:
recommend = 3
Y[Y <= recommend] = 0
Y[Y > recommend] = 1
n_pos = (Y == 1).sum()
n_neg = (Y == 0).sum()
print(f'{n_pos} positive samples and {n_neg} negative samples.')

2853 positive samples and 575 negative samples.


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,
                                                    random_state=42)

In [ ]:
print(len(Y_train), len(Y_test))

2742 686


In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=1.0, fit_prior=True)
clf.fit(X_train, Y_train)

MultinomialNB()

In [ ]:
prediction_prob = clf.predict_proba(X_test)
print(prediction_prob[0:10])

[[7.50487439e-23 1.00000000e+00]
 [1.01806208e-01 8.98193792e-01]
 [3.57740570e-10 1.00000000e+00]
 [1.00000000e+00 2.94095407e-16]
 [1.00000000e+00 2.49760836e-25]
 [7.62630220e-01 2.37369780e-01]
 [3.47479627e-05 9.99965252e-01]
 [2.66075292e-11 1.00000000e+00]
 [5.88493563e-10 9.99999999e-01]
 [9.71326867e-09 9.99999990e-01]]


In [ ]:
prediction = clf.predict(X_test)
print(prediction[:10])

[1. 1. 1. 0. 0. 0. 1. 1. 1. 1.]


In [ ]:
accuracy = clf.score(X_test, Y_test)
print(f'The accuracy is: {accuracy*100:.1f}%')

The accuracy is: 71.6%


# Trash!

In [ ]:

# find repeated Zip-code count
s = set()
# s = []
count = 0
# print(s)
for i in collective_data:
  # print(i[6])
  if i[6] not in s:
    s.add(i[6])
    # s.append([i[6],1])
  else:
    count += 1

print("Reapeted Zip-code count:")
print(count)
print(s)
print(len(s))




In [ ]:
# find repeated Zip-code
zip_code_count = {}
count = 0
print(zip_code_count)
for i in collective_data:
  if i[4] not in zip_code_count:  ##########
    zip_code_count[i[4]] = 1  ###############
  else:
    zip_code_count[i[4]] = zip_code_count[i[4]] + 1  ###################
    count += 1

print("Reapeted Zip-code count:")
print(count)
print(zip_code_count)
print(len(zip_code_count))

In [ ]:

  # p1 = (user_id_count[i[0]]['y'] + 1) /  (user_id_count[i[0]]['count'] + 2)
  # p2 = (gender_count[i[1]]['y'] + 1) /  (gender_count[i[1]]['count'] + 2)
  # p3 = (age_count[i[2]]['y'] + 1) /  (age_count[i[2]]['count'] + 2)
  # p4 = (occupation_count[i[3]]['y'] + 1) /  (occupation_count[i[3]]['count'] + 2)
  # try:
  #   p5 = (movie_id_count[i[4]]['y'] + 1) /  (movie_id_count[i[4]]['count'] + 2)
  # except:
  #   p5 = 1/2
  # p6 = (genres_count[i[5]]['y'] + 1) /  (genres_count[i[5]]['count'] + 2)



In [ ]:
# # find repeated Zip-code count
# s = set()
# count = 0
# # print(s)
# for i in user_data:
#   if i[4] not in s:  #############
#     s.add(i[4])  ######################
#   else:
#     count += 1

# print("Reapeted Zip-code count:")
# print(count)